This notebook will extract information from all of our DDOS csv files and place information into a large samples/features matrix. The matrix will look something like:
```
SOURCE_IP | VICTIM_IP | <Protocol>TRAFFIC_FROM_SRC_TO_DEST ...
```
Where `<Protocol>` can be ICMP, UDP, TCP, and a few others. 

In [330]:
import pandas as pd 
import numpy as np 
import csv as csv 
import glob 
import pickle
from datetime import datetime 
import matplotlib.pyplot as plt 
%matplotlib inline

In [331]:
victimIP = "71.126.222.64"
to_drop = ['<->', 'rel_start']
names = ['src', '<->', 'dest', 'in_frames', 'in_bytes', 'out_frames', 'out_bytes', 'total_frames', 'total_bytes', 'rel_start', 'duration']

In [343]:
# Begin dataframe by importing the total data, which will have all IPs
protocol_names = ['total' + "_" + name for name in names]
print(protocol_names)
protocol_to_drop = ['total' + "_" + name for name in to_drop]
ddos_df = pd.read_csv('resources/smallddos_total.tshark.csv', header=None, skiprows=5, names=protocol_names)
ddos_df = ddos_df[ddos_df.total_src != victimIP]
ddos_df['label'] = 'ddos'
ddos_df['total_out_bytes_rate'] = 0
ddos_df['total_out_frames_rate'] = 0

for i, row in ddos_df.iterrows():
    if row.total_out_bytes > 0 and row.total_duration != 0:
        ddos_df.loc[i, 'total_out_bytes_rate'] = row.total_out_bytes / row.total_duration
    if row.total_out_bytes > 0 and row.total_duration != 0:
        ddos_df.loc[i, 'total_out_frames_rate'] = row.total_out_frames / row.total_duration    
ddos_df = ddos_df.drop(ddos_df.tail(1).index)
ddos_df = ddos_df.drop(protocol_to_drop, axis=1)


['total_src', 'total_<->', 'total_dest', 'total_in_frames', 'total_in_bytes', 'total_out_frames', 'total_out_bytes', 'total_total_frames', 'total_total_bytes', 'total_rel_start', 'total_duration']


In [344]:
ddos_files = glob.glob('resources/smallddos*')
for i in range(len(ddos_files)):
    protocol = ddos_files[i][20:-11] # e.g., icmp, tcp, total, etc
    print("PROTOCOL : {0}".format(protocol))
    if protocol == 'total' or protocol == 'tcpPORTS':
        pass
    else:
        # Prefix the column names with the name of the protocol, e.g.:
        #     icmp_src icmp_dest icmp_out_bytes ...
        protocol_names = [protocol + "_" + name for name in names]
        protocol_to_drop = [protocol + "_" + col for col in to_drop]
        
        # Read the csv
        df = pd.read_csv(ddos_files[i], header=None, skiprows=5, names=protocol_names)
        
        # Drop rows we don't need, including those where the src IP is the victim
        df = df.drop(df.tail(1).index)
        # TODO: Change so that pairs where victimIP is src is labelled
        # victim_response instead of being dropped.
        df = df[df[protocol + "_src"] != victimIP]
        df = df.drop(protocol_to_drop, axis=1)
        
        protocol_names.remove(protocol + "_src")
        protocol_names.remove(protocol + "_<->")
        protocol_names.remove(protocol + "_dest")
        protocol_names.remove(protocol + "_rel_start")
        protocol_names.remove(protocol + "_in_frames")
        protocol_names.remove(protocol + "_in_bytes")
        protocol_names.remove(protocol + "_total_frames")
        protocol_names.remove(protocol + "_total_bytes")
           
        ddos_df[protocol + '_out_bytes_rate'] = 0 # icmp_out_bytes_rate
        ddos_df[protocol + '_out_frames_rate'] = 0# icmp_out_frames_rate
        for protocol_name in protocol_names:
            ddos_df[protocol_name] = 0
        
        for row_index, row in df.iterrows():
            protocol_source_ip = row[protocol + '_src']
            idx = ddos_df.loc[ddos_df['total_src'] == protocol_source_ip].index
            for protocol_name in protocol_names:
                ddos_df.loc[idx[0], protocol_name] = row[protocol_name]
            if row[protocol + '_out_bytes'] > 0 and row[protocol + '_duration'] != 0:
                ddos_df.loc[idx[0], protocol + '_out_bytes_rate'] = row[protocol + '_out_bytes'] / row[protocol + '_duration']
            if row[protocol + '_out_frames'] > 0 and row[protocol + '_duration'] != 0:
                ddos_df.loc[idx[0], protocol + '_out_frames_rate'] = row[protocol +'_out_frames'] / row[protocol + '_duration']

PROTOCOL : icmp
PROTOCOL : tcp
PROTOCOL : tcpPORTS
PROTOCOL : total
PROTOCOL : with_tcpsyn


In [334]:
# Now let's import Carlos' tcpPORTS data and add it as a feature.
with open('ddos_tcpPORTS.pickle', 'rb') as handle:
    ddos_ports_dict = pickle.load(handle)
# with open('passive_tcpPORTS.pickle', 'rb') as handle:
#     passive_ports_dict = pickle.load(handle)

In [335]:
# Create a new column in the dataframe which contains the
# number of ports used to attack the victim:
ddos_df['tcp_ports_used'] = 0
not_found = 0
for ip_pair in ddos_ports_dict:
    idx = ddos_df.loc[ddos_df['total_src'] == ip_pair[0]].index
    try:
        ddos_df.loc[idx[0], 'tcp_ports_used'] = len(ddos_ports_dict[ip_pair])
    except:
        not_found = not_found + 1

In [336]:
# Now we drop the features that we're not interested in -- 
# that is, all data not having to do with rates. 
bad_items = ['total_in_frames','total_in_bytes',\
             'total_out_frames','total_out_bytes',\
             'total_total_frames','total_total_bytes',\
             'total_duration','icmp_out_frames',\
             'icmp_out_bytes','icmp_duration','tcp_out_frames',\
             'tcp_out_bytes','tcp_duration','with_tcpsyn_out_frames',\
             'with_tcpsyn_out_bytes','with_tcpsyn_duration']
ddos_df = ddos_df.drop(bad_items, axis=1)

In [339]:
cols = ddos_df.columns.values
cols[0] = 'src'
cols[1] = 'dst'
ddos_df.columns = cols
ddos_df.head()

,src,dst,label,total_out_bytes_rate,total_out_frames_rate,icmp_out_bytes_rate,icmp_out_frames_rate,tcp_out_bytes_rate,tcp_out_frames_rate,with_tcpsyn_out_bytes_rate,with_tcpsyn_out_frames_rate,tcp_ports_used
1,195.216.1.86,71.126.222.64,ddos,7996.798079,148.088853,4709.376326,78.489605,3699.229327,77.067278,3699.229327,77.067278,37
2,195.52.240.138,71.126.222.64,ddos,6958.368734,127.442651,4205.607477,70.093458,2753.346080,57.361377,2753.346080,57.361377,27
4,209.210.127.200,71.126.222.64,ddos,8236.994220,142.100193,7080.924855,118.015414,1230.453730,25.634453,1230.453730,25.634453,10
5,167.46.3.209,71.126.222.64,ddos,5860.023725,108.738632,3202.846975,53.380783,2886.597938,60.137457,2886.597938,60.137457,23
6,217.238.32.123,71.126.222.64,ddos,5798.984772,107.614213,3273.185061,54.553084,2694.386694,56.133056,2694.386694,56.133056,23


In [342]:
ddos_df.describe()

,total_out_bytes_rate,total_out_frames_rate,icmp_out_bytes_rate,icmp_out_frames_rate,tcp_out_bytes_rate,tcp_out_frames_rate,with_tcpsyn_out_bytes_rate,with_tcpsyn_out_frames_rate,tcp_ports_used
count,4849.000000,4849.000000,4849.000000,4849.000000,4849.000000,4849.000000,4849.000000,4849.000000,4849.000000
mean,3120.493438,52.635030,3068.822314,51.147039,183.751532,3.828284,183.751532,3.828284,1.152403
std,36199.604533,603.359150,36643.763825,610.729397,850.146041,17.711594,850.146041,17.711594,3.917159
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1470.588235,24.721879,1423.487544,23.724792,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2057.281162,34.431874,1987.224982,33.120416,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2645.914397,44.444444,2528.507685,42.141795,0.000000,0.000000,0.000000,0.000000,0.000000
max,1800000.000000,30000.000000,1800000.000000,30000.000000,27428.571429,571.428571,27428.571429,571.428571,37.000000


In [340]:
# Pickle the data for later use:
with open('ddos_df_pickled.pickle', 'wb') as handle:
    pickle.dump(ddos_df, handle)

### Miscellaneo

In [218]:
# Verifying that all src IPs are distinct...
def anydup(thelist):
  seen = set()
  for x in thelist:
    if x in seen: return True
    seen.add(x)
  return False

print(anydup(ddos_df.total_src))

False


In [225]:
print(anydup(ddos_ports_dict))

False


In [341]:
ddos_df.columns.values

array(['src', 'dst', 'label', 'total_out_bytes_rate',
       'total_out_frames_rate', 'icmp_out_bytes_rate',
       'icmp_out_frames_rate', 'tcp_out_bytes_rate', 'tcp_out_frames_rate',
       'with_tcpsyn_out_bytes_rate', 'with_tcpsyn_out_frames_rate',
       'tcp_ports_used'], dtype=object)

### Passive Dataset